# Airnow Monitoring Site ESDR Device Cache

Downloads the ESDR devices representing the known Airnow monitoring site locations and caches as a JSON file.  Having a local cache is useful for speeding up uploads.  Used by cocalc scripts, and in user-facing visualizations.

Reports to stat.createlab.org as `Airnow Monitoring Site ESDR Device Cache`.

In [0]:
import json, os, dateutil, re, requests, subprocess, datetime, glob, stat

from dateutil import rrule, tz, parser

In [0]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb


def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']

    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())


exec_ipynb('./python-utils/utils.ipynb')
exec_ipynb('./python-utils/esdr-library.ipynb')
exec_ipynb('./airnow-common.ipynb')

In [0]:
MIRROR_TIME_PERIOD_SECS = 60 * 5   # every 5 minutes

STAT_SERVICE_NAME = 'Airnow Monitoring Site ESDR Device Cache'
STAT_HOSTNAME = 'airnow'
STAT_SHORTNAME = 'airnow-monitoring-site-esdr-device-cache'

ESDR_MONITORING_SITE_LOCATION_DEVICES_JSON_FILENAME = 'esdr_monitoring_site_location_devices.json'

In [0]:
Stat.set_service(STAT_SERVICE_NAME)

In [0]:
esdr = None
airnow_product = None

In [0]:
def get_airnow_product():
    global esdr, airnow_product
    if not esdr:
        esdr = Esdr('esdr-auth-airnow-uploader.json', user_agent='esdr-library.py['+STAT_SERVICE_NAME+']')
    if not airnow_product:
        # esdr.create_product('AirNow', 'AirNow', 'EPA and Sonoma Tech', 'Real-time feeds from EPA/STI AirNow')
        airnow_product = esdr.get_product_by_name('AirNow')
    return airnow_product

def refresh_esdr_monitoring_site_devices_cache():
    global esdr, airnow_product
    if not esdr:
        esdr = Esdr('esdr-auth-airnow-uploader.json', user_agent='esdr-library.py['+STAT_SERVICE_NAME+']')
    if not airnow_product:
        airnow_product = get_airnow_product()

    # get all ESDR devices belonging to the Airnow product, dealing with multiple pages of data if necessary
    devices = []
    while True:
        try:
            response = esdr.api('GET', '/api/v1/devices', {'where':'productId='+str(airnow_product['id']), 'fields':'id,name,serialNumber', 'offset':len(devices)})
            if 'data' in response and 'rows' in response['data']:
                new_rows = response['data']['rows']
                devices.extend(new_rows)
                if len(devices) == response['data']['totalCount'] or len(new_rows) <= 0:
                    break;
            else:
                raise Exception("No data in response when fetching ESDR devices")
        except requests.HTTPError as e:
            Stat.warning('Failed to refresh %s due to error: %s' % (ESDR_MONITORING_SITE_LOCATION_DEVICES_JSON_FILENAME, str(e)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            break;
        except Exception as e:
            Stat.warning('Failed to refresh %s due to error: %s' % (ESDR_MONITORING_SITE_LOCATION_DEVICES_JSON_FILENAME, str(e)), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
            break;

    # create a map, which maps serial number to device id and name
    esdr_device_map = {}
    for device in devices:
        esdr_device_map[device['serialNumber']] = {'id':device['id'], 'name':device['name']}

    # now write the JSON file
    json_dest = AirnowCommon.DATA_DIRECTORY + '/' + ESDR_MONITORING_SITE_LOCATION_DEVICES_JSON_FILENAME
    tmp = json_dest + '.tmp' + str(os.getpid())
    os.makedirs(os.path.dirname(tmp), exist_ok=True)
    with open(tmp, 'w') as json_file:
        json.dump(esdr_device_map, json_file, sort_keys=True)
    os.rename(tmp, json_dest)

    # make the JSON file readable by everyone
    os.chmod(json_dest, stat.S_IREAD | stat.S_IWRITE | stat.S_IRGRP | stat.S_IROTH)

    Stat.info('Successfully updated %s ' % ESDR_MONITORING_SITE_LOCATION_DEVICES_JSON_FILENAME, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

In [0]:
def run():
    Stat.up('Downloading Airnow monitoring site ESDR devices and caching to %s...' % ESDR_MONITORING_SITE_LOCATION_DEVICES_JSON_FILENAME, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    refresh_esdr_monitoring_site_devices_cache()
    Stat.up('Done downloading Airnow monitoring site ESDR devices and caching to %s' % ESDR_MONITORING_SITE_LOCATION_DEVICES_JSON_FILENAME, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

def run_forever():
    while True:
        run()
        sleep_until_next_period(MIRROR_TIME_PERIOD_SECS)

run_forever()